In [1]:
import calendar
import time
import datetime
import pandas as pd
import numpy as np
from itertools import compress

# 排课

## 定义解析时间的函数

In [87]:
def parse_time(time):
    parsed = datetime.datetime.strptime(time, '%Y-%m-%d %H:%M')
    return parsed

## 定义课节

In [88]:
class Classes:
    def __init__(self, name, duration):
        self.name = name
        self.duration = duration

In [89]:
python_1on1 = Classes('Python 1on1', duration=40)

## 定义检测冲突的函数

In [90]:
def check_conflict(previous, proposal, rest):
    '''previous: list of tuples enveloping start time and end time
       current: list of tuples enveloping start time and end time'''
    buffle = datetime.timedelta(minutes = rest)
    checked = [not (prvs[0]>= proposal[1] + buffle or prvs[1] <= proposal[0] - buffle) for prvs in previous]
    conflict = list(compress(range(len(checked)), checked))
    conf_class = [previous[i] for i in conflict]
    return checked, conf_class

## 检测老师可排课时间

In [96]:
def check_available(available, proposal):
    checked = [avlb[0] <= proposal[0] and avlb[1] >= proposal[1] for avlb in available]
    occupation = list(compress(range(len(checked)), checked))
    return checked, occupation

In [100]:
start_1 = '2019-02-03 19:00'
end_1 = '2019-02-03 21:00'
start_2 = '2019-02-05 19:00'
end_2 = '2019-02-05 21:00'
available = [(start_1, end_1), (start_2, end_2)]
available = [tuple(map(parse_time, (a, b))) for (a, b) in available]
# parse_time(end_1).strftime('%Y-%m-%d %H:%M')
# [tuple(map(parse_time, (a, b))) for (a, b) in []]

In [103]:
check_available(available, (datetime.datetime(2019, 2, 3, 19,20), datetime.datetime(2019, 2, 3, 20,0)))
check_conflict(available, (datetime.datetime(2019, 2, 3, 19,20), datetime.datetime(2019, 2, 3, 20,0)), 10)

([True, False],
 [(datetime.datetime(2019, 2, 3, 19, 0),
   datetime.datetime(2019, 2, 3, 21, 0))])

## 定义讲师

In [179]:
class Lecturer:
    def __init__(self, name, available, rest = 10, schedule = {}):
        '''available: list of tuples enveloping start time and end time'''
        self.name = name
        self.__reg_available = available.copy()
        self.available = available
        self.rest = rest
        self.schedule = schedule
    def get_reg_available(self):
        return self.__reg_available
    
    def init_appointment(self, your_class, student, start):
        self.classes = your_class
        available_parse = [tuple(map(parse_time, (a, b))) for (a, b) in self.available]
        proposal = (parse_time(start), parse_time(start) + datetime.timedelta(minutes = your_class.duration))
        availability, occupation = check_available(available_parse, proposal)
        if any(availability):
            previous = np.concatenate(list(self.schedule.values())) if bool(self.schedule) else list(self.schedule.values())
            previous = [tuple(map(parse_time, (a, b))) for (a, b) in previous]
            conflict, conf_class = check_conflict(previous, proposal, self.rest)
            if any(conflict):
                conf_class = tuple(map(lambda x: x.strftime('%Y-%m-%d %H:%M'), conf_class))
                print('老师在{}-{}这段时间内有课，而且请给老师{}分钟休息时间，另行安排!'.format(conf_class[0], conf_class[1], self.rest))
            else:
                end = parse_time(start) + datetime.timedelta(minutes = self.classes.duration)
                print('没问题，该时间段内老师可排课！')
                if student not in self.schedule.keys():
                    self.schedule[student] = [(start, end.strftime('%Y-%m-%d %H:%M'))]
                else:
                    self.schedule[student].extend([(start, end.strftime('%Y-%m-%d %H:%M'))])
                # 剩余时间
                available_parse = [tuple(map(parse_time, (a, b))) for (a, b) in self.available]
                for i in occupation:
                    insertee = [(available_parse[i][0], parse_time(start)),
                                (end, available_parse[i][1])]
                    insertee = list(filter(lambda x: x[1] > x[0], insertee))
                    insertee = [tuple(map(lambda x: x.strftime('%Y-%m-%d %H:%M'), (a, b))) for (a, b) in insertee]
                    del self.available[i]
                    self.available[i:i] = insertee
        else:
            print('老师有安排了，请考虑以下时间段：\n{}'.format(self.available))
    def change_appointment(self, student, which, prop_start):
        prop_end = parse_time(prop_start) + datetime.timedelta(minutes = self.classes.duration)
        available_parse = [tuple(map(parse_time, (a, b))) for (a, b) in self.available]
        availability, occupation =  check_available(available_parse, (parse_time(prop_start), prop_end))
        if any(availability):
            previous = np.concatenate(list(self.schedule.values())) if bool(self.schedule) else list(self.schedule.values())
            previous = [tuple(map(parse_time, (a, b))) for (a, b) in previous]
            conflict, conf_class = check_conflict(previous, (parse_time(prop_start), prop_end), self.rest)
            conf_class = tuple(map(parse_time, conf_class))
            if any(conflict):
                print('老师在{}-{}这段时间内有课，而且请给老师{}分钟休息时间，另行安排!'.format(conf_class[0], conf_class[1], self.rest))
            else:
                print('没问题，该时间段内老师可排课！')
                self.available.append(self.schedule[student][which])
                self.available = sorted([x for x in self.available])
                self.schedule[student][which] = (prop_start, prop_end.strftime('%Y-%m-%d %H:%M'))

        else:
            print('老师有安排了，请考虑以下时间段：\n{}'.format(self.available))

定义讲师李海

In [180]:
Lihai = Lecturer(name = '李海', available = [(start_1, end_1),
                                             (start_2, end_2)])
print('讲师姓名：{}'.format(Lihai.name))
print('讲师可排课时间：{}'.format(Lihai.available))

讲师姓名：李海
讲师可排课时间：[('2019-02-03 19:00', '2019-02-03 21:00'), ('2019-02-05 19:00', '2019-02-05 21:00')]


初次预约课程。

In [181]:
Lihai.init_appointment(python_1on1, 'Libai', start= '2019-02-03 19:30')
print('{}老师，你的课程表如下：\n{}\n请提前做好准备！'.format(Lihai.name, Lihai.schedule))
print('老师的剩余可用时间为\n{}'.format(Lihai.available))

没问题，该时间段内老师可排课！
李海老师，你的课程表如下：
{'Libai': [('2019-02-03 19:30', '2019-02-03 20:10')]}
请提前做好准备！
老师的剩余可用时间为
[('2019-02-03 19:00', '2019-02-03 19:30'), ('2019-02-03 20:10', '2019-02-03 21:00'), ('2019-02-05 19:00', '2019-02-05 21:00')]


给第二个学生预约课程：

In [182]:
Lihai.init_appointment(python_1on1, 'DuPu', start= '2019-02-03 20:20')
print('{}老师，你的课程表如下：\n{}\n请提前做好准备！'.format(Lihai.name, Lihai.schedule))
print('老师的剩余可用时间为\n{}'.format(Lihai.available))

没问题，该时间段内老师可排课！
李海老师，你的课程表如下：
{'Libai': [('2019-02-03 19:30', '2019-02-03 20:10')], 'DuPu': [('2019-02-03 20:20', '2019-02-03 21:00')]}
请提前做好准备！
老师的剩余可用时间为
[('2019-02-03 19:00', '2019-02-03 19:30'), ('2019-02-03 20:10', '2019-02-03 20:20'), ('2019-02-05 19:00', '2019-02-05 21:00')]


杜同学要排第二节课

In [184]:
Lihai.init_appointment(python_1on1, 'DuPu', start= '2019-02-05 20:20')
print('{}老师，你的课程表如下：\n{}\n请提前做好准备！'.format(Lihai.name, Lihai.schedule))
print('老师的剩余可用时间为\n{}'.format(Lihai.available))
# Lihai.schedule['DuPu'][0]
# available = Lihai.available.copy()
# available.append(Lihai.schedule['DuPu'][0])
# available

没问题，该时间段内老师可排课！
李海老师，你的课程表如下：
{'Libai': [('2019-02-03 19:30', '2019-02-03 20:10')], 'DuPu': [('2019-02-03 20:20', '2019-02-03 21:00'), ('2019-02-05 20:20', '2019-02-05 21:00')]}
请提前做好准备！
老师的剩余可用时间为
[('2019-02-03 19:00', '2019-02-03 19:30'), ('2019-02-03 20:10', '2019-02-03 20:20'), ('2019-02-05 19:00', '2019-02-05 20:20')]


杜同学要调课

In [185]:
Lihai.change_appointment('DuPu', 0, '2019-02-05 19:00')
print('{}老师，你的课程表如下：\n{}\n请提前做好准备！'.format(Lihai.name, Lihai.schedule))
print('老师的剩余可用时间为\n{}'.format(Lihai.available))

没问题，该时间段内老师可排课！
李海老师，你的课程表如下：
{'Libai': [('2019-02-03 19:30', '2019-02-03 20:10')], 'DuPu': [('2019-02-05 19:00', '2019-02-05 19:40'), ('2019-02-05 20:20', '2019-02-05 21:00')]}
请提前做好准备！
老师的剩余可用时间为
[('2019-02-03 19:00', '2019-02-03 19:30'), ('2019-02-03 20:10', '2019-02-03 20:20'), ('2019-02-05 19:00', '2019-02-05 20:20'), ('2019-02-03 20:20', '2019-02-03 21:00')]


In [211]:
i = 3
i -= 1
i

2

In [213]:
available = sorted([x for x in Lihai.available])
print(available)
for i in range(1, len(available)):
    if available[i][0] == available[i-1][1]:
        available[i-1] = (available[i-1][0], available[i][1])
        del available[i]
        print()
        print(i)
print(available)

[('2019-02-03 19:00', '2019-02-03 19:30'), ('2019-02-03 20:10', '2019-02-03 20:20'), ('2019-02-03 20:20', '2019-02-03 21:00'), ('2019-02-05 19:00', '2019-02-05 20:20')]
2


IndexError: list index out of range

In [ ]:
class Schedule:
    def __init__(self, year, month):
        self.year = year
        self.month = month
    def term(self, lenghth):
        cale = calendar.Calendar()
        self.term_length = lenghth
        month_span = self.term_length // 28 +1
        schedule = cale.monthdatescalendar(year=self.year, month=self.month)
        schedule = pd.DataFrame(schedule,columns = calendar.weekheader(6).strip().split('    ')).iloc[:4]
#         schedule = pd.DataFrame(columns = calendar.weekheader(6).strip().split('    '))
        for i in range(1,month_span):
            schedule_i = cale.monthdatescalendar(year=self.year, month=self.month + i)
            schedule_i = pd.DataFrame(schedule_i,columns = calendar.weekheader(6).strip().split('    '))
            schedule = schedule.append(schedule_i)
            schedule.reset_index(drop=True, inplace=True)
        return schedule

In [ ]:
Schedule(2019, 1).term(124)

In [ ]:
18*7

In [ ]:
class Lecturer:
    def __init__(self, name, available):
        self.name = str(name),
        self.available = available
    def end(self, duration):
        '''calculate where his class will ends'''
        ends = []
        for i in self.available:
            ends.append(i+ datetime.timedelta(minutes = duration))
        return ends
    def assign(self, schedual):
        '''assign his available time to te schedual'''
        for i in self.available:
            date = i.year
            loc = schedual.loc('')

In [ ]:
Lihai = Lecturer(name='李海', available=[datetime.datetime(2019, 2, 3, 19, 0), datetime.datetime(2019, 2, 5, 10, 0)])
datetime.datetime(2019, 1, 23, 19, 0, 0)

In [ ]:
schedual['Mon'][1] == datetime.datetime(2019, 2, 4)

# 尝试群发学习计划

In [ ]:
import pandas as pd
import itchat as ic

In [ ]:
ic.auto_login()

获取单个用户的ID

In [ ]:
userID = ic.search_friends(name='Udacity-Tyelcie')[0]['UserName']
userID

获取群ID

In [ ]:
chatroomID = ic.search_chatrooms('测试')[0]['UserName']
chatroomID

发文字消息

In [ ]:
ic.send('https://shimo.im/sheet/xwJ8FkJhUcUKeXyM/', toUserName=userID)

发图片消息

In [ ]:
ic.send_image('../stobserver/test.jpg', toUserName=chatroomID)

## 简单粗暴地抓取石墨文档表格

把要抓的内容选中，Ctrl+C，然后用`pd.read_clipboard()`从剪贴板读取数据。

In [ ]:
students = pd.read_clipboard(sep='\t')
students.head(3)

如果要群发消息，注意修改任务序列和消息模板：

In [ ]:
task_seq = 'Task 1'
msg_template = 'Hello，{}同学，接下来三天你需要完成的任务是：\n\n{}'
def send_task_group(task_seq, msg_template, students_df):
    for i in range(1, len(students_df)):
        student = students_df['姓名'][i]
        userID = ic.search_friends(name=student)[0]['UserName']
        personal_task = students_df[task_seq][i]
        msg = msg_template.format(student, personal_task)
        ic.send(msg, userID)

In [ ]:
task_seq = 'Task 1'
msg_template = 'Hello，{}同学，接下来三天你需要完成的任务是：\n\n{}'
send_task_group(task_seq, msg_template, students)

如果需要单独发消息，注意修改学生姓名、任务序列和消息模板：

In [ ]:
student = 'Udacity-Tyelcie'
task_seq = 'Task 1'
msg_template = 'Hello，{}同学，接下来三天你需要完成的任务是：\n\n{}'

def send_task_personal(student_name, task_seq, msg_template, students_df):
    stu_pos = list(filter(lambda x: students_df['姓名'][x] == student_name, students_df.index.tolist()))[0]
    msg = msg_template.format(student_name, students_df[task_seq][stu_pos])
    userID = ic.search_friends(name=student_name)[0]['UserName']
    ic.send(msg, userID)

In [ ]:
send_task_personal('Udacity-Tyelcie', task_seq, msg_template, students)